In [ ]:
import time
import duckdb
import torch

import db_queries
import main




# these are the files to which the sql database will be saved
db_file_c = duckdb.connect("/cluster/work/coss/anmusso/victoria/loaded_data/loaded_comments.db")
db_file_s = duckdb.connect("/cluster/work/coss/anmusso/victoria/loaded_data/loaded_submissions.db")



(We could consider changing extract_comments to also add to existing tables)
Before running any code adjust base_paths in other files
Load comments into duckdb and get embeddings 

In [ ]:


main.load('S', '05', '2020') #loads submissions of may 2020
#main.load('C', '05', '2020') #loads comments of may 2020
#main.load('S', '06', '2020')
#main.load('C', '06', '2020')

In [ ]:
#run to check number of submissions/comments loaded
start = time.time()
print("C5: \n", db_file_c.sql("SELECT Count(*) FROM C_05_2020;"))
print("S5: \n", db_file_s.sql('SELECT Count(*) FROM S_05_2020;'))
print("C6: \n", db_file_c.sql('SELECT Count(*) FROM C_06_2020;'))
print("S6: \n", db_file_s.sql('SELECT Count(*) FROM S_06_2020;'))
print("time to compute: ", time.time()-start)

Calculate embeddings: (uncomment line)
available models: 'all-mpnet-base-v2' , 'all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1'
will automatically use gpu if available
use main.py with batch system to choose GPU

In [ ]:
t,m,y = 'S','06','2020' #type month and year
model_name = 'all-MiniLM-L6-v2'


start = time.time()
num = main.embedd(m, y, t, model_name=model_name) #returns number of embeddings calculated

with open('time_tracking.txt', 'a') as f: #loggs time into file for comparison
    f.write(f"set: {m+t+y}, size:{num} model:{model_name}, gpu:{torch.cuda.get_device_name(0)} , time:{time.time()-start} \n")


Retrieve all entry numbers for certain subreddits within given table

In [ ]:
t,m,y = 'S','06','2020' #type month and year
subr = ['Republican', 'democrats','healthcare', 'Feminism', 'nra', 'education', 'climatechange',
            'politics','progressive', 'The_Donald','TrueChristian','Trucks','teenagers','AskMenOver30',
            'backpacking','news','BlackLivesMatter','racism','news','usa','DefundPoliceNYC']

subr_nums = db_queries.get_subreddit_numbers(table_name = main.table_id(t,m,y), sql_db = db_file_s, subreddits = subr)
print(len(subr_nums))

load embeddings of retrieved numbers

In [ ]:
M_embedd = main.fetch_embeddings(num_list = subr_nums, table_name=main.table_id(t,m,y), model_name= 'all-MiniLM-L6-v2')
print(M_embedd.shape)
#print(M_embedd[0])

now we can umap the embeddings

In [ ]:
import embedding_graphs.UMAP_embeddings as umap

atr = ['title', 'subreddit']
atr_df = db_queries.get_attributes(attributes=atr, numbers=subr_nums, table_name=main.table_id(t,m,y), sql_db=db_file_s)
print(atr_df)
umap.show_umap(M_embedd,atr_df,'umap try 1')